Get started with Docker
===


## Installation

To install docker you can simply follow the instruction you can find at this website [https://docs.docker.com/v17.09/engine/installation/](https://docs.docker.com/v17.09/engine/installation/)

When you finished you can check that everything works correctly by opening a terminal and typing "docker run hello-world" (you may need to add sudo for macOS and Linux), then you should see something like this

![Hello World](images/helloworld.png)

If you get that you're ready to proceed with the example below

How to use Docker container
===
![Example applicatio](images/dockerExample.png)

We want to create the code for the 3 different actors above and we want assign _Catalog_ and _Sensor_ to two different container, while instead _Monitor_ will run in the "usual" way. 
Before creating the container let's see how the different actors behave in order to have a better understanding. 


Sensor
---

The script _sensor.py_ is a simple REST client for a temperature and umidity sensor. When the script is launched a put request will be sent to the catalog in order to the _Catalog_ to notify that the sensor is alive and which are his settings (IP address, port, accepted methods). This notification will happen periodically to ensure that the _Catalog_ stays updated. The settings are stored in a file called _settings.json_. The code is written below

In [ ]:
import cherrypy
import requests
import json
import random
class SensorREST(object):
    exposed=True
    def __init__(self):
        self.settings=json.load(open('settings.json'))
        self.settings['commands']=['hum','temp']
        requests.put(self.settings['catalog'],data=json.dumps(self.settings))
    def GET(self,*uri,**params):
        if len(uri)!=0:
            if uri[0]=='hum':
                value=random.randint(60,80)
            if uri[0]=='temp':
                value=random.randint(10,25)
            output={'deviceID':self.settings['ID'],str(uri[0]):value}
            return json.dumps(output)
        else:
            return json.dumps(self.settings)
    def pingCatalog(self):
        requests.put(self.settings['catalog'],data=json.dumps(self.settings))

if __name__ == '__main__':
    conf={
        '/':{
                'request.dispatch':cherrypy.dispatch.MethodDispatcher(),
                'tool.session.on':True
        }
    }
    s=SensorREST()
    cherrypy.config.update({'server.socket_host': '0.0.0.0','server.socket_port': s.settings['port']})	
    cherrypy.tree.mount(s,'/',conf)
    cherrypy.engine.start()
    while True:
        print('sleeping')
        s.pingCatalog()
        time.sleep(10)
    cherrypy.engine.exit()

Catalog
---
The script _catalog.py_ is another REST client, his job is to keep and updated list of the available devices and their setting and to trovide it to toher actor that may need it (in our case _Monitor_). Everytime the _Catalog_ receives a request from a _Sensor_ it will add it to the list of the devices and will store the timestamp of that request. This list is periodically controlled to check if the last timestamp of each of this devices respects a threshold, if the timestamp is too "old" the device will be removed from the list. The settings are stored in a file called _settings.json_. The code for the _Catalog_ is below

In [ ]:
import cherrypy
import requests
import json 
import time
class Catalog(object):
    def __init__(self):
        self.devices=[]
        self.actualTime=time.time()
    def addDevice(self,devicesInfo):
        self.devices.append(devicesInfo)
    def updateDevice(self,deviceID,devicesInfo):
        for i in range(len(self.devices)):
            device=self.devices[i]
            if device['ID']==deviceID:
                self.devices[i]=devicesInfo
    def removeDevices(self,deviceID):
        for i in range(len(self.devices)):
            device=self.devices[i]
            if device['ID']==deviceID:
                self.devices.pop(i)
    def removeInactive(self):
        self.actualTime=time.time()
        for device in self.devices:
            if self.actualTime-device['last_update']>4:
                self.devices.remove(device)

class CatalogREST(object):
    exposed=True
    def __init__(self,clientID):
        self.ID=clientID
        self.catalog=Catalog()
    def GET(self,*uri,**params):
        output={'devices':self.catalog.devices,"updated":self.catalog.actualTime}
        return json.dumps(output)
    def PUT(self):
        body=cherrypy.request.body.read()
        json_body=json.loads(body.decode('utf-8'))
        if not any(d['ID']==json_body['ID'] for d in self.catalog.devices):
            last_update=time.time()
            json_body['last_update']=last_update
            self.catalog.addDevice(json_body)
            output=f"Device with ID {json_body['ID']} has been added"
            print (output)
            return output
        else:
            last_update=time.time()
            json_body['last_update']=last_update
            self.catalog.updateDevice(json_body['ID'],json_body)
            return json_body

    def DELETE(self,*uri):
        self.catalog.removeDevices(uri[0])
        output=f"Device with ID {uri[0]} has been removed"
        print (output)


if __name__ == '__main__':
    catalogClient=CatalogREST('Catalog')
    conf={
        '/':{
                'request.dispatch':cherrypy.dispatch.MethodDispatcher(),
                'tool.session.on':True
        }
    }
    cherrypy.config.update({'server.socket_host': '0.0.0.0','server.socket_port': 8080})
    cherrypy.tree.mount(catalogClient,'/',conf)
    cherrypy.engine.start()
    while True:
        catalogClient.catalog.removeInactive()
        time.sleep(5)
    cherrypy.engine.exit()

Monitor
---
The script _monitor.py_ it's a simple script to monitor retrieve the value of the available devices. It's able to get the list of available devices from the _Catalog_ and it will show the data coming from the chosen devices. The code is below


In [ ]:
import requests
import json
import time
class Viewer(object):
    """docstring for Viewer"""
    def __init__(self):
        self.catalogInfo=json.load(open("settings.json"))
        self.devices=self.getDevices()

    def getDevices(self):
        response=requests.get(self.catalogInfo['catalogURL']).json()
        print('List of available devices obtained')
        return  response['devices']

    def listDevices(self):
        print('This are the available devices:')
        for device in self.devices:
            print(device['ID'])
        idSelected=input("Which device to you want to monitor (r to refresh  q to quit)\n")
        if idSelected!='q':
            if idSelected=='r':
                self.getDevices()
                self.listDevices()
            else:
                idSelected=int(idSelected)
                self.monitor([x for x in self.devices if x['ID']==idSelected][0])
        else:
            exit()

    def monitor(self,device):
        print("Temp (C)\tHum(%)")
        end_time=time.time()+4
        while time.time()<end_time:
            temp=requests.get(device["IP"]+':'+str(device['port'])+'/temp').json()['temp']
            hum=requests.get(device["IP"]+':'+str(device['port'])+'/hum').json()['hum']
            print(str(temp)+'\t\t'+str(hum),end="\r")
            time.sleep(0.5)
        self.listDevices()

if __name__ == '__main__':
    v=Viewer()
    v.listDevices()

Dockerfile creation
===
In order to create a container for _Catalog_ and _Monitor_ we need two define two _Dockerfile_ that specify their settings: the file to use, the library to install, the command to execute etc. As we did before with the code we will analyze each actor separately

Requirements definition
---
The first thing to do before creating the dockerfile is to create a file calle _requirements.txt_ containing all the library that we need for our script. To do this there are 2 main ways:  

__First method__ 

For each of the library we import in our script we must run in the terminal the following command _"pip3 show <name of the library\>"_ (in some cases you can use pip instead of pip3) and we should see something like this

![pip show](images/pipshow.png)

form this result we need the _Name_ and the _Version_, which we will write in the file _requirements.txt_, in this case it would be

~~~
CherryPy==18.1.2
~~~

We must do this operation for all the library that are not "standard" adding a new line in the file for each of them (for example we can avoid it for the library _json_ and _time_)

__Second method__

The second method is the easiest to use but it require to install pipreqs by running  on the terminal the comand _"pip3 install pipreqs"_ (also in this case sometimes pip is enough). Once the installation is finished you can create the file _requirements.txt_ buy running the following command _"pipreqs <path of the folder with the scripts\>"_

Sensor Dockerfile
---
So the first thing to do is creating _requirements.txt_. Once we've done that we can can create a new file called _Dockerfile_ with no extension (no .txt no .py etc).
The first thing to write in the file is which language we're going to use, in our case it's Python 3 so we're going to write

~~~
FROM python:3
~~~
The next thing to do is to _ADD_ each file we're going to use specifing the path of origin and the destination. In the case of the sensor the file used are _sensor.py_, _settings.json_ and _requirements.txt_ so we will write

~~~
#ADD <origin> <destination>
ADD sensor.py /
ADD settings.json /
ADD requirements.txt /
~~~

After that we want to install all the library specified in the file _requirements.txt_. To do this we write

~~~
RUN pip3 install -r requirements.txt
~~~

The last thing we need to do is specify which comand to execute to start our script, so in this case

~~~
#CMD <comma-separated list of command>
CMD ["python3","./sensor.py"]
~~~

So at the end our dockerfile for _Sensor_ will look like this

~~~
FROM python:3
ADD sensor.py /
ADD settings.json /
ADD requirements.txt /
RUN pip3 install -r requirements.txt
CMD ["python3","./sensor.py"]
~~~

Catalog Dockerfile
---
The process is similar to the one explained above, at the end we should have something like 

~~~
FROM python:3
ADD catalog.py /
ADD requirements.txt /
RUN pip3 install -r requirements.txt
CMD ["python3","./catalog.py"]
~~~


Build and run and image inside a container
===

Build
---

To build the image of the docker we need to:

- Open the terminal
- go in the folder where the Dockerfile is located
- type "docker build . -t <tagForTheImage\>"

After doing that docker will start to download the needed version of the python and the libraries we specified in the file _requirements.txt_.  
For our case we need to do that bot for the catalog and the sensor.


Run
---

To run a container we simply need to type on the terminal this

~~~
docker run  --name <NameOfContainer> -p <localPort>:<dockerPort>  <imageTag>
~~~

If you want to run the container in background you can add the option -d before the image tag.

We can do this for both the _Catalog_ and the _Sensor_ but we must pay attention to specify **TWO DIFFERENT** local port in order to be able to talk to cboth of them from localhost